# With the RDS Library
---

RDS provides you with a powerful tool to access and manage data. Our API can quickly and efficiently filter the data and perform computations on it on the fly. It can also retrieve important metadata on each aspect of the data for you to perform more detailed analysis. To run this notebook, packages needed for generating the charts must be installed:

In [ ]:
!pip install -r requirements.txt

By utilizing a **Select** query, we are able to get the specific records of data we are interested in, along with performing standard computations, grouping, and ordering of that data at the same time. In the initial example below, we are interested in data revolving around COVID-19 deaths in Ohio, so we first create a connection to that data set through the `DataProduct` (To view other dataset, browse our catalog). With this data, we want to create a simple line plot of the number of deaths for the first 2 weeks of March. We specify the columns of data we want as the date of occurrence and as a computation that sums the deaths. Grouping by date ensures that we only have one record per date with the sum of deaths for that date. We then proceed to order the records and filter them to get the month and the amount of records we wanted.

If we are interested in more extensive information around any of the columns, we could check the metadata contained within the results returned by the query.

In [ ]:
from rds import DataProduct
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#Connect to the Ohio data set
dp = DataProduct('https://covid19.richdataservices.com', 'us_oh', 'oh_doh_cases')
#Query for the number of deaths over the first 2 weeks of March
results = dp.select(cols=['date_stamp', 'deaths:sum(cnt_death)'], groupby=['date_stamp'], orderby=['date_stamp'], where=['date_stamp>=2020-03-01'], limit=14)

#Plug in the data and build our line plot
df = pd.DataFrame(results.records, columns = results.columns)
sns.set(rc={'figure.figsize':(30, 10)})
sns.lineplot(data=df, x=df.columns[0], y=df.columns[1])
plt.show()

Utilizing the various options available through RDS, you can specify the data returned to be exactly what you need.

In this next example, we are now making use of a **Tabulate** query with multiple dimensions so that we can see the amount of deaths for each county. If you think about a tabulation table, the **'dims'** would be the rows and columns and the **'measure'** would be the value to fill in each cell. We then proceed with filtering and ordering the data returned like before. Setting the parameter `inject=True` fills in any coded values with their labels. Variables with classifications that specify these codes can be found in the metadata.

In [ ]:
from rds import DataProduct
from urllib.request import urlopen
import json
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

#Connect to the Ohio data set
dp = DataProduct('https://covid19.richdataservices.com', 'us_oh', 'oh_doh_cases')
#Query for the number of deaths for every county in Ohio
results = dp.tabulate(dims=['us_county_fips'], measure=['Deaths:sum(cnt_death)'], orderby=['us_county_fips'])
df = pd.DataFrame(results.records, columns = results.columns)

#Getting the structure of the map
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

#Plug in the data and build our choropleth map
fig = px.choropleth(df, geojson=counties, locations=df.columns[0], color='Deaths', color_continuous_scale="Reds", range_color=(0, 100), scope="usa", labels={'Deaths'})
fig.update_geos(fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, width=1000, height=500)
fig.show()

# Without RDS
---

Getting and organizing the data into a form you can work with is generally tedious and complicated. Below we demonstrate what you would need to do to get a single data set and format that data to be able to create the choropleth map you saw above.

The first thing we have to do is get the data and filter it for exactly what we want. Let'ss see what our dataframe looks like after we do just that.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Get the data
data = pd.read_csv('https://coronavirus.ohio.gov/static/COVIDSummaryData.csv')
df = pd.DataFrame(data)
#Filter the data
df = df[['County','Death Count']]
df = df.groupby(['County']).sum().reset_index()
df = df.sort_values('County')
print(df)

Well that doesn't look right. Seems that we understandably assumed the "Death Counts" would be and integer, but we actually have to transform it into integers ourselves. Let's try that.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Get the data
data = pd.read_csv('https://coronavirus.ohio.gov/static/COVIDSummaryData.csv')
df = pd.DataFrame(data)
#Filter the data
df = df[['County','Death Count']]
#Transform the data
df['Death Count'] = df['Death Count'].astype(int)
df = df.groupby(['County']).sum().reset_index()
df = df.sort_values('County')
print(df)

It appears that not all values in the "Death Counts" column are in in a valid integer format! It turns out to be the "Grand Total" row, which isn't even wanted for our choropleth map so we can just drop it from the dataframe.

In [ ]:
from urllib.request import urlopen
import matplotlib.pyplot as plt
import json
import plotly.express as px
import seaborn as sns
import pandas as pd

#Get the data
data = pd.read_csv('https://coronavirus.ohio.gov/static/COVIDSummaryData.csv')
df = pd.DataFrame(data)
#Filter the data
df = df[['County','Death Count']]
#Clean the data
df = df[df['County'] != 'Grand Total']
#Transform the data
df['Death Count'] = df['Death Count'].astype(int)
df = df.groupby(['County']).sum().reset_index()
df = df.sort_values('County')
print(df)

#Getting the structure of the map
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

#Plug in the data and build our choropleth map
fig = px.choropleth(df, geojson=counties, locations=df.columns[0], color='Death Count', color_continuous_scale="Reds", range_color=(0, 100), scope="usa", labels={'Deaths'})
fig.update_geos(fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, width=1000, height=500)
fig.show()

Our dataframe looks good now, so why isn't our choropleth map working with it?

Unfortunately, the choropleth map requires the FIPS codes for each county where we only have the names. To get this work, we will have to convert each county name into its FIPS code and try again. If we were to just use RDS, we could switch from name to FIPS by setting `inject=True` in our query. That would've been so much easier.

In [ ]:
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
import plotly.express as px
import seaborn as sns
import pandas as pd
import requests
from bs4 import BeautifulSoup

#Get the data
data = pd.read_csv('https://coronavirus.ohio.gov/static/COVIDSummaryData.csv')
df = pd.DataFrame(data)
#Filter the data
df = df[['County','Death Count']]
#Clean the data
df = df[df['County'] != 'Grand Total']
#Transform the data
df['Death Count'] = df['Death Count'].astype(int)
df = df.groupby(['County']).sum().reset_index()
df = df.sort_values('County')

#Scrape for the county FIPS codes
html = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county").text
soup = BeautifulSoup(html,"lxml")
table = soup.find('table', {"class":"wikitable sortable"}).find('tbody')

#Replace the county names with their FIPS code
state = ''
for df_row in df.iterrows():
    county = df_row[1]['County']
    for tb_row in table.find_all('tr'):
        cells = tb_row.findChildren('td')
        if len(cells) == 3:
            state = str(cells[2].text).strip()
        
        if len(cells) >= 2 and county in cells[1].text and state == 'Ohio':
            df = df.replace(county, str(cells[0].text).strip())
            break

#Getting the structure of the map
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

#Plug in the data and build our choropleth map
fig = px.choropleth(df, geojson=counties, locations=df.columns[0], color='Death Count', color_continuous_scale="Reds", range_color=(0, 100), scope="usa", labels={'Deaths'})
fig.update_geos(fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, width=1000, height=500)
fig.show()

Since the data from our CSV is missing the FIPS codes, we have to scrape them from wikipedia. Even after that, we still need to replace every single county name with its FIPS code in our dataframe before it'll work. The result of this is some ugly loops as well as slow code.

As you can see, RDS handles all of the hard work of organizing, transforming, and cleaning the data, allowing you to focus on the presentation of the data instead.

## See our [Catalog](./covid_19_catalog.ipynb) for a list of available data sets, as well as the metadata on any variables and their classifications.